# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [78]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *
import utils

##### CONNECTING THE DATABASE


In [5]:
try:
    connection = psycopg2.connect(host = '127.0.0.1', 
                                  dbname = 'sparkifydb', 
                                  user = 'student', 
                                  password = 'student')
except psycopg2.Error as e:
    print('ERROR: Could not connect to DB')
    print(e)

##### GETTING A CURSOR TO THE DATABASE

In [6]:
try:
    cursor = connection.cursor()
except psycopg2.Error as e:
    print('ERROR: Could not get curso to DB')
    print(e)    

In [9]:
utils.get_files('data/log_data/', '.json')

['/home/workspace/data/log_data/2018/11/2018-11-30-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-11-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-23-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-24-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-25-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-20-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-15-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-01-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-03-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-13-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-10-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-28-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-26-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-04-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-14-events.json',
 '/home/workspace/data/lo

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`

In [11]:
song_files = get_files('data/song_data', '.json')

- Select the first song in this list


In [12]:
filepath = song_files[0]

- Read the song file and view the data

In [15]:
songs_df = pd.read_json(filepath, lines = True)
songs_df.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARD7TVE1187B99BFB1,NaN,California - LA,NaN,Casual,218.93179,1,SOMZWCG12A8C13C480,I Didn't Mean To,0


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `songs_df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [17]:
# MUST BE A LIST 
# NEITHER the original songs_df
# NOR the numpy array return by .values
song_data = list(*songs_df[['song_id', 'title', 'artist_id', 'year', 'duration']].values)

In [18]:
song_data

['SOMZWCG12A8C13C480', "I Didn't Mean To", 'ARD7TVE1187B99BFB1', 0, 218.93179]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [19]:
cursor.execute(songs_table_insert, song_data)
connection.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `songs_df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [20]:
songs_df.sample()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARD7TVE1187B99BFB1,NaN,California - LA,NaN,Casual,218.93179,1,SOMZWCG12A8C13C480,I Didn't Mean To,0


In [23]:
artist_data = list(*songs_df[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']].values)
artist_data

['ARD7TVE1187B99BFB1', 'Casual', 'California - LA', nan, nan]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [25]:
cursor.execute(artists_table_insert, artist_data)
connection.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [27]:
log_files = get_files('data/log_data', '.json')

In [28]:
filepath = log_files[0] # SINGLE DAY OF LOG

In [30]:
log_df = pd.read_json(filepath, lines = True)
log_df.head(3)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Stephen Lynch,Logged In,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",PUT,NextSong,1.540992e+12,829,Jim Henson's Dead,200,1543537327796,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,91
1,Manowar,Logged In,Jacob,M,0,Klein,247.56200,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Shell Shock,200,1543540121796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
2,Morcheeba,Logged In,Jacob,M,1,Klein,257.41016,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Women Lose Weight (Feat: Slick Rick),200,1543540368796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [132]:
# Filter records by NextSong action
nextsong_df = log_df.query('page == "NextSong"')

In [133]:
# Convert the ts timestamp column to datetime
nextsong_df['ts'] = pd.to_datetime(nextsong_df['ts'], unit = 'ms')

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [134]:
# CHEKING THE INSERT STATEMENT FOR THE ORDER OF THE COLUMNS
time_table_insert.split('\n')

['',
 '                     INSERT INTO time(start_time, hour, day, week, month, year, weekday)',
 '                                 VALUES(to_timestamp(%s), %s, %s, %s, %s, %s, %s)',
 '']

In [154]:
# Extract the timestamp, hour, day, week of year, month, year, and weekday from the ts column 
start_time = nextsong_df['ts'].copy()
hour = timestamp.dt.hour
day = timestamp.dt.day
week = timestamp.dt.weekofyear
month = timestamp.dt.month
year = timestamp.dt.year
weekday = timestamp.dt.weekday

# Set time_data to a list containing these values in order
time_vars = [start_time, hour, day, week, month, year, weekday]
# time_data = [series.values.tolist() if idx != 0 else series for idx, series in enumerate(time_vars) ]
time_data = [series.values.tolist() for series in time_vars ]

In [155]:
# ENSURING THAT THE COLUMN LABELS WILL BE IN THE RIGHT ORDER
column_labels = [utils.var_name(col, globals())[0] for col in time_vars]
column_labels

['start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday']

In [166]:
#Create a dataframe, time_df, containing the time data for this file by combining column_labels and time_data into a dictionary and converting this into a dataframe
time_df = pd.DataFrame(dict(zip(column_labels, time_data)))
time_df.head()

,start_time,hour,day,week,month,year,weekday
0,1543537327796000000,0,30,48,11,2018,4
1,1543540121796000000,1,30,48,11,2018,4
2,1543540368796000000,1,30,48,11,2018,4
3,1543540625796000000,1,30,48,11,2018,4
4,1543540856796000000,1,30,48,11,2018,4


In [169]:
time_df['start_time'] = (time_df['start_time'] / 1000000).astype(int)

In [170]:
time_df.head()

,start_time,hour,day,week,month,year,weekday
0,1543537327796,0,30,48,11,2018,4
1,1543540121796,1,30,48,11,2018,4
2,1543540368796,1,30,48,11,2018,4
3,1543540625796,1,30,48,11,2018,4
4,1543540856796,1,30,48,11,2018,4


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [ ]:
for idx, row in time_df.iterrows():
    try:
        cursor.execute(time_table_insert, list(row))
        connection.commit()
    except psycopg2.Error as e:
        print("ERROR inserting data")
        print(e)
        connection.rollback()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [174]:
log_df.columns

Index(['artist', 'auth', 'firstName', 'gender', 'itemInSession', 'lastName',
       'length', 'level', 'location', 'method', 'page', 'registration',
       'sessionId', 'song', 'status', 'ts', 'userAgent', 'userId'],
      dtype='object')

In [175]:
# Nota Bene: We dropped duplicates & missing values to abide by the constraints of user_id being PRIMARY KEY 
user_df = log_df[['userId', 'firstName', 'lastName', 'gender', 'level']].drop_duplicates().dropna()

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [176]:
for i, row in user_df.iterrows():
    try:
        cursor.execute(users_table_insert, list(row))
        connection.commit()
    except psycopg2.Error as e:
        print("ERROR inserting data")
        print(e)
        connection.rollback()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [177]:
songplays_table_insert = ("""
                          INSERT INTO songplays(start_time, user_id, level, song_id , 
                                                artist_id , session_id , location , user_agent) 
                                         VALUES(to_timestamp(%s), %s, %s, %s, %s, %s, %s, %s )
                          """)
                    

In [190]:
for index, row in log_df.iterrows():
    
    values = (row['song'], row['artist'], row['length'])

    # get songid and artistid from song and artist tables
    try:
        cursor.execute(song_select, values)
        results = cursor.fetchone()

    except psycopg2.Error as e:
        print('ERROR: Could not select data')
        print(e)
            
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    # Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to songplay_data
    timestamp = row['ts']
    user_id = row['userId']
    level = row['level']
    session_id = row['sessionId']
    location = row['location']
    user_agent = row['userAgent']
    
    songplay_data = (timestamp, user_id, level, songid, artistid, session_id , location, user_agent)    

    cursor.execute(songplays_table_insert, songplay_data)
    connection.commit()

DataError: invalid input syntax for integer: ""
LINE 4: ...              VALUES(to_timestamp(1543544407796), '', 'paid'...
                                                             ^


There seems to be missing values... 

artist           58
auth              0
firstName        12
gender           12
itemInSession     0
lastName         12
length           58
level             0
location         12
method            0
page              0
registration     12
sessionId         0
song             58
status            0
ts                0
userAgent        12
userId            0
dtype: int64

Run `test.ipynb` to see if you've successfully added records to this table.

In [185]:
len(songplay_data)

6

# Close Connection to Sparkify Database

In [77]:
cursor.close()
connection.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.